## Colab Setting

In [ ]:
# Rendering problems in Colab 
# https://stackoverflow.com/questions/63250935/nameerror-name-base-is-not-defined-while-running-open-ai-gym-in-google-colab
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet

!pip3 install box2d-py
!pip3 install gym[Box_2D]

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()
#=========================================================#

import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

# why unwrapped
# https://stackoverflow.com/questions/53836136/why-unwrap-an-openai-gym
env = gym.make('CartPole-v0').unwrapped

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 784 kB of archives.
After this operation, 2,271 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.10 [784 kB]
Fetched 784 kB in 1s (927 kB/s)
Selecting previously unselected package xvfb.
(Reading database ... 155639 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.10_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.10) ...
Setting up xvfb (2:1.19.6-1ubuntu4.10) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/

## DDPG

### Model

The output of the actor network is an exact action, instead of a probability distribution over actions. That's deterministic policy.

In [ ]:
class Actor(nn.Module):
  def __init__(self, state_dim, action_dim, max_action_value):
    super(Actor, self).__init__()
    self.fc1 = nn.Linear(state_dim, 64)
    self.fc2 = nn.Linear(64, 64)
    self.fc3 = nn.Linear(64, action_dim)
    self.max_action_value = max_action_value
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    return self.max_action_value * torch.tanh(self.fc3(x))

class Critic(nn.Module):
  def __init__(self, state_dim, action_dim):
    super(Critic, self).__init__()
    self.fc1 = nn.Linear(state_dim + action_dim, 64)
    self.fc2 = nn.Linear(64, 64)
    self.fc3 = nn.Linear(64, 1)
  
  def forward(self, state, action):
    input = torch.cat([state, action], 1)
    q = F.relu(self.fc1(input))
    q = F.relu(self.fc2(q))
    return self.fc3(q)

The critic takes in state and action, outputs the q value.

### Replay Buffer

Use just numpy array with some counters to implement Replay Buffer.

I think it is more convenient than using namedtuple and deque.

In [ ]:
class ReplayBuffer(object):
  def __init__(self, state_dim, action_dim, capacity=int(1e6), batch_size=64):
    self.capacity = capacity
    self.batch_size = batch_size
    self.counter = 0

    self.state = np.zeros((capacity, state_dim))
    self.action = np.zeros((capacity, action_dim))
    self.next_state = np.zeros((capacity, state_dim))
    self.reward = np.zeros((capacity, 1))
    self.done = np.zeros((capacity, 1))

  def add(self, state, action, next_state, reward, done):
    index = self.counter % self.capacity

    self.state[index] = state
    self.action[index] = action
    self.next_state[index] = next_state
    self.reward[index] = reward
    self.done[index] = done

    self.counter += 1
  
  def sample(self, batch_size=64):
    # min handle the situation when buffer is not full
    indices_range = min(self.counter, self.capacity)
    indices = np.random.randint(0, indices_range, size=batch_size)
    return(
      torch.Tensor(self.state[indices]).to(device),
      torch.Tensor(self.action[indices]).to(device),
      torch.Tensor(self.next_state[indices]).to(device),
      torch.Tensor(self.reward[indices]).to(device),
      torch.Tensor(self.done[indices]).to(device)
    )
  
  def can_sample(self):
    return self.counter > self.batch_size

### Ornstein-Uhlenbeck process

A method to encourage exploration according to the DDPG paper since in DDPG there is no epsilon-greedy policy.

This part is taken from https://github.com/vitchyr/rlkit/blob/master/rlkit/exploration_strategies/ou_strategy.py

In [ ]:
class OUNoise(object):
  def __init__(self, action_space, mu=0.0, theta=0.15, max_sigma=0.3, min_sigma=0.3, decay_period=100000):
    self.mu           = mu
    self.theta        = theta
    self.sigma        = max_sigma
    self.max_sigma    = max_sigma
    self.min_sigma    = min_sigma
    self.decay_period = decay_period
    self.action_dim   = action_space.shape[0]
    self.low          = action_space.low
    self.high         = action_space.high
    self.reset()
      
  def reset(self):
    self.state = np.ones(self.action_dim) * self.mu
      
  def evolve_state(self):
    x  = self.state
    dx = self.theta * (self.mu - x) + self.sigma * np.random.randn(self.action_dim)
    self.state = x + dx
    return self.state
  
  def get_action(self, action, t=0): 
    ou_state = self.evolve_state()
    self.sigma = self.max_sigma - (self.max_sigma - self.min_sigma) * min(1.0, t / self.decay_period)
    return np.clip(action + ou_state, self.low, self.high)

### Agent

In [ ]:
from copy import deepcopy

class DDPGAgent:
  def __init__(self, state_dim, action_dim, max_action_value, actor_learning_rate=1e-4, critic_learning_rate=1e-3, memory=int(1e6), tau=1e-3):
    self.tau = tau
    self.action_dim = action_dim
    self.max_action_value = max_action_value

    self.actor = Actor(state_dim, action_dim, max_action_value).to(device)
    self.actor_target = deepcopy(self.actor)
    self.acotr_optimizer = optim.Adam(self.actor.parameters(), lr=actor_learning_rate)

    self.critic = Critic(state_dim, action_dim)
    self.critic_target = deepcopy(self.critic)
    self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=critic_learning_rate)

    self.replay_buffer = ReplayBuffer(state_dim, action_dim, capacity=memory)

  # A way to increase exploration suggested by some blogs
  # the idea is add some Gaussian noise to the action
  # An alternative of OU-noise
  def process_action(self, action, exploration_noise=0.1):
    action += np.random.normal(0, self.max_action_value * exploration_noise, size=self.action_dim)
    action = action.clip(-self.max_action_value, self.max_action_value)
    return action
  
  def select_action(self, state):
    state = torch.FloatTensor(state).to(device)
    with torch.no_grad():
      action = self.actor(state).cpu().data.numpy() #.flatten
    
    action = self.process_action(action)

    return action

  def soft_update(self, online_model, target_model, tau=1e-3):
    for online_param, target_param in zip(online_model.parameters(), target_model.parameters()):
      target_param.data.copy_(tau * online_param.data + (1.0 - tau) * target_param.data)

  def update(self, gamma=0.99):
    states, actions, next_states, rewards, dones = self.replay_buffer.sample() # Tensors

    next_actions = self.actor_target(next_states).detach()
    next_q_values = self.critic_target(next_states, next_actions).detach()
    target_q_values = rewards + gamma * next_q_values * (1 - dones)
    predict_q_values = self.critic(states, actions)

    # Critic loss is the TD-error between the Q value of current and next timestep
    critic_loss = F.mse_loss(predict_q_values, target_q_values)
    self.critic_optimizer.zero_grad()
    critic_loss.backward()
    self.critic_optimizer.step()

    # Actor loss is the mean of Q values given by the critic
    actor_loss = - self.critic(states, self.actor(states)).mean()
    self.acotr_optimizer.zero_grad()
    actor_loss.backward()
    self.acotr_optimizer.step()

    self.soft_update(self.critic, self.critic_target, self.tau)
    self.soft_update(self.actor, self.actor_target, self.tau)
  
  def step(self, state, action, next_state, reward, done):
    self.replay_buffer.add(state, action, next_state, reward, done)
    if self.replay_buffer.can_sample():
      self.update()

In the final training, using OU-Noise can let the agent learn faster in the beginning, but it is always hard to converge. Using Gaussian Noise, on the other hand, will be slower for learning but the rate of convergence is higher.

In [ ]:
from collections import deque
from datetime import datetime

def train(agent, noise, num_episodes=2000, max_timestep=1000):
  scores = []
  scores_window = deque(maxlen=100)
  start_time = datetime.now().replace(microsecond=0)
  #timestep = 0   # used for OUNoise decaying

  for i_episode in range(1, num_episodes+1):
    state = env.reset()
    #noise.reset()
    score = 0
    for t in range(max_timestep):
      action = agent.select_action(state)
      #action = noise.get_action(action, timestep)

      next_state, reward, done, _ = env.step(action)
      agent.step(state, action, next_state, reward, done)

      state = next_state
      score += reward

      #timestep += 1
      
      if done:
        break

    scores_window.append(score)
    scores.append(score)

    print('\rEpisode {}\tScore: {:.2f}\tAverage Score: {:.2f}'.format(i_episode, score, np.mean(scores_window)), end='')
    if i_episode % 100 == 0:
      print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
    if np.mean(scores_window) >= 200.0:
      print('\nEnvironment solved in {:d} episodes!\tAverage Score {:.2f}'.format(i_episode-100, np.mean(scores_window)))
      break
  
  end_time = datetime.now().replace(microsecond=0)
  print("Total training time  : ", end_time - start_time)

  return scores

In [ ]:
env = gym.make('LunarLanderContinuous-v2')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action_value = env.action_space.high[0]

agent = DDPGAgent(state_dim, action_dim, max_action_value, actor_learning_rate=1e-4, critic_learning_rate=1e-3)
noise = OUNoise(env.action_space)
ddpg_scores = train(agent, noise, num_episodes=2000)

Episode 100	Average Score: -583.05
Episode 200	Average Score: -273.12
Episode 300	Average Score: -159.59
Episode 400	Average Score: -87.22
Episode 500	Average Score: -80.85
Episode 600	Average Score: -60.64
Episode 700	Average Score: -49.90
Episode 800	Average Score: -49.01
Episode 900	Average Score: -15.95
Episode 1000	Average Score: 114.83
Episode 1100	Average Score: 176.67
Episode 1116	Score: 253.45	Average Score: 201.06
Environment solved in 1016 episodes!	Average Score 201.06
Total training time  :  1:09:09


## Conclusion

My implementation of DDPG can solved LunarLanderContinuous environement within pretty decent timesteps. However, the actual consuming time is quite long. This may due to 4 networks are used. However, the trained policy is not always converged. This point out the weakness of DDPG: it is hard to extend to high dimensions problem.

From my point of view, the poor convergence may caused by the exploration mechanism. In the DDPG paper, OU-Noise is used. Some will suggest using Gaussian Noise. But either of them can not outperform stochastic hehaviour policy regarding exploration.

What's more, considering the final performance in continuous environment, using 4 networks is too expensive. The goal of using 4 networks is to maintain the stability of the underlying DQN algorithm. I personally feel that in DDPG, most of the efforts is spent on making DQN work in continuous environment, which to me like a detour.

Policy Gradient is much straightforward in solving continuous environment. It inherently can output continous action, and has a good exploration ability without extra efforts of adding other functions. Also, the backpropagation in policy gradient makes more sense than DQN, and it will not encounters the problem of overoptimistic caused by max operation. The things we want to fix for policy gradient is that, we want to have a better way to update our networks, to make each update is guaranteed to improve theoretically. What's more, we want the algorithm to be more sample-efficent.

All these problems is solved by PPO, which is the most popular algorithm nowadays.